In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertConfig, BertModel
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
[WinError 1455] The paging file is too small for this operation to complete. Error loading "D:\CODE\Python\SentimentalAnalysis\venv\lib\site-packages\torch\lib\cudnn_cnn_infer64_8.dll" or one of its dependencies.

In [0]:

# Load training data from "train.csv"
train_df = pd.read_csv("synthetic_train.csv")

# Load validation data from "val.csv"
val_df = pd.read_csv("synthetic_val.csv")

label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['sentiment'])
val_df['label_encoded'] = label_encoder.transform(val_df['sentiment'])

# Use the BertTokenizer for Vietnamese
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [0]:

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True)

train_df['input_ids'] = train_df['sentence'].apply(tokenize_text)
val_df['input_ids'] = val_df['sentence'].apply(tokenize_text)

bert_config = BertConfig.from_pretrained('bert-base-multilingual-cased')

input_layer = Input(shape=(128,), dtype=tf.int32)
embedding_layer = Embedding(input_dim=bert_config.vocab_size, output_dim=bert_config.hidden_size)(input_layer)
pooling_layer = GlobalAveragePooling1D()(embedding_layer)
output_layer = Dense(units=len(label_encoder.classes_), activation='softmax')(pooling_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

train_input_ids = np.array(train_df['input_ids'].to_list())
val_input_ids = np.array(val_df['input_ids'].to_list())

In [1]:

history = model.fit(
    train_input_ids, train_df['label_encoded'].values,
    validation_data=(val_input_ids, val_df['label_encoded'].values),
    epochs=10, batch_size=128
)

# Save the model
model.save('sentiment_model')

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
[WinError 1455] The paging file is too small for this operation to complete. Error loading "D:\CODE\Python\SentimentalAnalysis\venv\lib\site-packages\torch\lib\cudnn_cnn_infer64_8.dll" or one of its dependencies.